# Goal: Predict vehicle sale prices (Sold_amount)

1. Highlight errors and concerns with the data

2. Feature selection / engineering. 
Warning: Cannot use AvgWholesale, AvgRetail, GoodWholesale, GoodRetail, TradeMin, TradeMax, PrivateMax. This will result in overfitting.

3. Different ways of evaluting and diagnosing the model.

In [ ]:
!pip3 install dataprep

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dataprep

from scipy.stats import ttest_1samp, ttest_ind
from dataprep.eda import create_report

In [ ]:
# Quicklook Function
def ql(df, rows=5):
    """
    DataFrame Quicklook
    
    Prints shape of dataframe as well as top or bottom X rows.
    Inputs:
    - df  : dataframe
    - rows: number of rows, default = 5
    - part: "head" or "tail", default = "head"
    
    Outputs:
    - top and bottom rows
    """
    
    # Print dataframe dimensions
    print(df.shape)
    print(df.head(rows))
    print(df.tail(rows))
    

In [ ]:
data = pd.read_csv('DatiumSample.rpt', delimiter="\t")

In [ ]:
# To preview the data
data.to_csv("raw_data.csv")

In [ ]:
df = data.copy()

In [ ]:
df.info()

### Noticed some columns had a lot of null values

In [ ]:
df.isnull().mean().round(4).mul(100).sort_values(ascending=False)

In [ ]:
# Noticed there are a lot of missing values in certain columns.
# Start with a crude method of removing any columns with less than 80% values
df.dropna(thresh=df.shape[0]*0.8, how='all', axis=1, inplace = True)

In [ ]:
# Noticed there were 4 missing values in the response variable
df.dropna(subset=["Sold_Amount"], inplace= True)

In [ ]:
# From the correlation matrix

#Drop the collinear categorical data.
df.drop(["MakeCode","FamilyCode", "DriveCode"], axis=1, inplace= True)
    # , "TareMass", "GrossVehicleMass"
    # ,"GrossCombinationMAss", "FuelCapacity", "WheelBase", "Height", "Length", "Width", "EngineSize"

#Drop the sparse columns
df.drop(["MonthGroup"], axis=1, inplace= True) # 73.3% is missing through "0".

#Drop the collinear continuous data
df.drop(["GoodKM", "AvgRetail", "GoodRetail", "PrivateMax", "NewPrice", "GoodWholesale"
         #, "AvgWholesale",  "TradeMin", "TradeMax"
         #, "Cylinders", "TorqueRPMTo", "AirpollutionRating", "OverallGreenStarRating"
        ], axis=1, inplace= True)

#Drop collinear with predictor 
#df.drop(["CO2Combined", "FuelUrban", "FuelExtraurban", "FuelCombined", "TowingBrakes", "TowingNoBrakes", "Torque"], axis=1, inplace= True)
#"CO2Urban", "CO2ExtraUrban", 

#Drop the constants
df.drop(["ImportFlag"], axis=1, inplace= True)

In [ ]:
df.describe()

In [ ]:
df = df[df["Sold_Amount"]>99]

In [ ]:
df["log_SequenceNum"] = np.log(df["SequenceNum"])
df["log_Age_Comp_Months"] = np.log(df["Age_Comp_Months"])
df["log_KM"] = np.log(df["KM"])
df["log_sold_amount"] = np.log(df["Sold_Amount"])


In [ ]:
df.drop(columns=["SequenceNum"], inplace=True)
df.drop(columns=["Age_Comp_Months"], inplace=True)
df.drop(columns=["KM"], inplace=True)
df.drop(columns=["Sold_Amount"], inplace=True)

In [ ]:
df["Compliance_Date"] = pd.to_datetime(df["Compliance_Date"])
df["Sold_Date"] = pd.to_datetime(df["Sold_Date"])

In [ ]:
create_report(df)

# Missing values

Little's MCAR test for Python:
https://www.kaggle.com/yassirarezki/handling-missing-data-mcar-mar-and-mnar-part-i

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(df.isna().transpose(),
            cmap="YlGnBu",
            cbar_kws={'label': 'Missing Data'})
plt.savefig("visualizing_missing_data_with_heatmap_Seaborn_Python.png", dpi=100)

#Missing values + descriptive statistics after


# Concerns with data

1. Data sparsity